In [74]:
from qm.qua import *
from qm import QuantumMachinesManager, SimulationConfig
from quam.components import BasicQuAM, SingleChannel
from quam.components.pulses import SquarePulse
from qualang_tools.units import unit
from QuAM_utilities import *

import os
import matplotlib.pyplot as plt
from matplotlib import patches
import time
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler, PatternMatchingEventHandler
import numpy as np
from numpy import array
from astropy.io import fits

u = unit() 

In [75]:
# silence the warnings?
import warnings
warnings.filterwarnings("ignore")

from IPython.display import clear_output

In [76]:
ip_address = '130.79.148.167'
qmm = QuantumMachinesManager(host=ip_address, log_level='DEBUG')

2024-12-20 21:46:18,595 - qm - DEBUG    - Probing gateway at: 130.79.148.167:80
2024-12-20 21:46:19,653 - qm - DEBUG    - Connection redirected from '130.79.148.167:80' to '130.79.148.167:10010'
2024-12-20 21:46:19,657 - qm - DEBUG    - Established connection to 130.79.148.167:10010
2024-12-20 21:46:19,658 - qm - DEBUG    - Gateway discovered at: 130.79.148.167:80
2024-12-20 21:46:19,798 - qm - INFO     - Performing health check
2024-12-20 21:46:19,806 - qm - INFO     - Health check passed


Configure the machine: define the channels and associate the pulses

In [77]:
# AOM RFs
test_rf = 100*u.kHz
test_rf = 0

# pulse variables
test_amplitude = 0.5
test_time = 100*u.us

strobo_periods = 3

gap1_time = 5*u.ms
gap2_time = 5*u.ms
imag_time = 50*u.ms
coll_time = 50*u.ms
delay_imaging_time = 250.15*u.ms

machine = BasicQuAM()
machine.channels['test'] = test = SingleChannel(opx_output=('con1', 7), intermediate_frequency=test_rf)

test.operations['high_pulse'] = high_pulse = SquarePulse(length=int(test_time), amplitude=test_amplitude)
test.operations['low_pulse'] = low_pulse = SquarePulse(length=int(test_time), amplitude=0)

qua_config = machine.generate_config()

In [78]:
with program() as streams_test:
    input_rf = declare_input_stream(int, name='input_rf_input_stream') # NB: check the type of the input

    with infinite_loop_():
        advance_input_stream(input_rf)
        update_frequency('test', input_rf, keep_phase=True)
        wait_for_trigger('test') 
        play('high_pulse', 'test')
        play('low_pulse', 'test')

In [79]:
with program() as IO_test:
    #input_rf = declare_input_stream(int, name='input_rf_input_stream') # NB: check the type of the input
    n = declare(int)

    with infinite_loop_():
        #advance_input_stream(input_rf)
        with for_(var=n, init=10000, cond=n<=200000, update=n+10000):
            assign(IO1, n)
            wait_for_trigger('test') 
            update_frequency('test', n, keep_phase=True)
            play('high_pulse', 'test')
            play('low_pulse', 'test')
            pause()

In [80]:
with program() as amplitudeonly_test:
    input_amp = declare_input_stream(fixed, name='input_amp_input_stream') # NB: check the type of the input

    with infinite_loop_():
        advance_input_stream(input_amp)
        play('high_pulse'*amp(input_amp), 'test')
        #play('high_pulse', 'test')
        play('low_pulse', 'test')

In [81]:
with program() as amplitudeonly_strobo:
    input_amp = declare_input_stream(fixed, name='input_amp_input_stream') # this must be a qua variable

    with infinite_loop_():
        advance_input_stream(input_amp)
        stroboscopic_amp_mod(('high_pulse','low_pulse'), 'test', strobo_periods, amp_mod=input_amp)

In [82]:
with program() as amplitude_frequency_strobo:
    input_amp = declare_input_stream(fixed, name='input_amp_input_stream') # this must be a qua variable
    input_rf = declare_input_stream(int, name='input_rf_input_stream')

    with infinite_loop_():
        advance_input_stream(input_amp)
        advance_input_stream(input_rf)
        update_frequency('test', input_rf/2, keep_phase=True)
        stroboscopic_amp_mod(('high_pulse','low_pulse'), 'test', strobo_periods, amp_mod=input_amp)

In [83]:
with program() as test_det_amp_optimization_streams:
    input_cool_double_rf = declare_input_stream(int, name='input_rf_input_stream')
    #input_repump_double_rf = declare_input_stream(int, name='repump_double_rf_input_stream')
    input_cool_amp_imag = declare_input_stream(fixed, name='cool_amp_imag_input_stream')
    #input_repump_amp_imag = declare_input_stream(fixed, name='repump_amp_imag_input_stream')
        
    with infinite_loop_():
        advance_input_stream(input_cool_double_rf)
        #advance_input_stream(input_repump_double_rf)
        advance_input_stream(input_cool_amp_imag)
        #advance_input_stream(input_repump_amp_imag)
        update_frequency('test', 50, keep_phase=True)
        wait_for_trigger('test')

        long_pulse('high_pulse', 'test', gap1_time + 3*coll_time) # try to change
        #long_pulse('repump_continuous_coll', 'AOM_single_repumper', gap1_time + coll_time)

        update_frequency('test', input_cool_double_rf, keep_phase=True)
        #update_frequency('AOM_double_repumper', input_repump_double_rf, keep_phase=True)

        long_pulse_amp_mod('high_pulse', 'test', gap2_time + 3*imag_time, amp_mod=input_cool_amp_imag) # try to change
        #long_pulse('repump_continuous_imag', 'AOM_single_repumper', gap2_time + imag_time)

        # delay for image
        #long_pulse('low_pulse', 'test', delay_imaging_time)
        #long_pulse('repump_low_pulse', 'AOM_double_repumper', delay_imaging_time)

        #long_pulse('high_pulse', 'test', imag_time) # try to change
        #long_pulse('repump_continuous_imag', 'AOM_single_repumper', delay_imaging_time + imag_time)

In [84]:
# upon image creation
class StreamsHandler(PatternMatchingEventHandler):
    def __init__(self, patterns, qm, opx_job):
        super().__init__(patterns=patterns, ignore_directories=True)
        self.qm = qm
        self.job = opx_job

    def on_created(self, event):
        # this functoin is called whenever a new "pattern" file is created
        
        clear_output(wait=True)
        print(f"Processing file: {event.src_path}")
        time.sleep(0.2) # wait for M-LOOP to write the file
        with open(event.src_path, 'r') as file:
            line = file.readline().strip()
            new_parameters = np.array(eval(line.split('=')[1].strip())) # take the argument after the "=" and evaluate the string as python variable
        print("The Dog has watched:", new_parameters) # for debugging

        new_rf, new_amp = new_parameters
        frac = new_amp/0.5
        self.job.push_to_input_stream('input_rf_input_stream', int(new_rf))
        self.job.push_to_input_stream('cool_amp_imag_input_stream', float(frac))
        #print("New frequency:", self.qm.get_io1_value())
        #self.job.resume()
        os.remove(event.src_path)
        #print("New values:")
        #print(f" cooler frequency: {cool_double_rf_imag}")
        #print(f" repumper frequency: {repump_double_rf_imag}")

In [85]:
# get the parameters from the exp_input file 
path_to_watch = r"C:\Users\EQM\Giovanni\OPXsetup\OPX_EQM\Optimization\imaging_parameters"
pattern = 'exp_input.txt'

if __name__ == '__main__':
    qm = qmm.open_qm(qua_config)
    job = qm.execute(test_det_amp_optimization_streams)
    observer = Observer()
    event_handler = StreamsHandler(patterns=[pattern], qm=qm, opx_job = job)
    observer.schedule(event_handler, path=path_to_watch, recursive=True)
    observer.start()

    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
    observer.join()

Processing file: C:\Users\EQM\Giovanni\OPXsetup\OPX_EQM\Optimization\imaging_parameters\exp_input.txt
The Dog has watched: [15.46490669  0.32114169]


In [40]:
qm = qmm.open_qm(qua_config)
job = qm.execute(amplitude_frequency_strobo)

2024-12-20 17:58:43,555 - qm - WARNING  - Open QM ended with warning 0: Opening a new Quantum Machine and closing Quantum Machines: 
2024-12-20 17:58:43,556 - qm - WARNING  - Open QM ended with warning 0: Quantum Machine qm-1734713914578 is canceling running job 1734512114521.
2024-12-20 17:58:43,567 - qm - INFO     - Sending program to QOP for compilation
2024-12-20 17:58:43,855 - qm - INFO     - Executing program


In [41]:
new_amplitude = 0.25
new_rf = int(20e3)
fact = new_amplitude/test_amplitude
print(fact)
job.push_to_input_stream('input_amp_input_stream', fact)
job.push_to_input_stream('input_rf_input_stream', new_rf)

0.5


In [12]:
job.execution_report()

Execution report for job 1734512114546
No errors

In [18]:
job.halt()

True

Notes on stream test:
- there are some synchronization issues
- sometimes i see an update of just the first value, then the job pauses
- IO READ TEST. The reading of the assigned value via the IO variable seems to work. In particular, we first see the update on the oscilloscope (new sequene from the OPX), and then we ge the reading from the output 
- STREAM ONLY TEST. The signal gets always updated, it does not stop.
- STREAM TEST. With the trigger, seems to work only for a couple of iterations...is it in conflict with the pause since they do similar things? BNC cable time is approx 50ns. Without the trigger works fine.

Suspects:
- The problem might be trigger+pause!
- trigger+get_io

TODO:
- try with amplitude modification
- try with a single channel but for the whole sequence 

In [ ]:
qm.close()
# qm.compile(det_optimization)